In [1]:
import pandas as pd
import numpy as np
import os
import random
import tensorflow as tf

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
import matplotlib.pyplot as plt
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.legacy import Adam

import wandb
from wandb.keras import WandbCallback
import numpy as np
import random
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
# wandb.login(key='bcbdf183b36d76416116c8af7c131cd84592f116')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/yanicewan/.netrc


True

In [2]:
wandb.init(project="Clinical", entity="yixuany")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yixuany. Use `wandb login --relogin` to force relogin


In [7]:
def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
wandb.init(project="Clinical", entity="yixuany")
# Configure wandb to log hyperparameters
config = wandb.config
config.epochs = 100
config.batch_size = 32
config.learning_rate = 0.001
config.decay = 1e-2
config.dropout = 0.2

# Set random seed
seed = random.randint(1, 200)
wandb.log({"seed": seed})
reset_random_seeds(seed)

# Define the model
model = Sequential()
model.add(Dense(128, input_shape=(149,), activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(config.dropout))
model.add(Dense(64, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(config.dropout))
model.add(Dense(50, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(config.dropout))
model.add(Dense(3, activation="softmax"))

# Compile the model
model.compile(Adam(learning_rate=config.learning_rate, decay=config.decay),
              "sparse_categorical_crossentropy",
              metrics=["sparse_categorical_accuracy"])

# Custom callback to log metrics to wandb
class CustomWandbCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        wandb.log({
            "epoch": epoch,
            "train_loss": logs["loss"],
            "train_accuracy": logs["sparse_categorical_accuracy"],
            "val_loss": logs["val_loss"],
            "val_accuracy": logs["val_sparse_categorical_accuracy"]
        })

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=config.epochs,
    validation_split=0.1,
    batch_size=config.batch_size,
    verbose=1,
    callbacks=[CustomWandbCallback()]
)

# Evaluate on test data
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# Make predictions and calculate metrics
test_predictions = model.predict(X_test)
test_label = to_categorical(y_test, 3)
true_label = np.argmax(test_label, axis=1)
predicted_label = np.argmax(test_predictions, axis=1)

# Calculate precision, recall, and F1-score
cr = classification_report(true_label, predicted_label, output_dict=True)

# Log final metrics to wandb
wandb.log({
    "test_accuracy": score[1],
    "precision": cr["macro avg"]["precision"],
    "recall": cr["macro avg"]["recall"],
    "f1_score": cr["macro avg"]["f1-score"]
})

wandb.finish()

Epoch 1/100
39/39 [==============================] - 1s 5ms/step - loss: 1.0359 - sparse_categorical_accuracy: 0.5711 - val_loss: 1.3493 - val_sparse_categorical_accuracy: 0.4234
Epoch 2/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8551 - sparse_categorical_accuracy: 0.6255 - val_loss: 0.8306 - val_sparse_categorical_accuracy: 0.5182
Epoch 3/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8562 - sparse_categorical_accuracy: 0.6158 - val_loss: 0.7162 - val_sparse_categorical_accuracy: 0.6715
Epoch 4/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8261 - sparse_categorical_accuracy: 0.6109 - val_loss: 0.6956 - val_sparse_categorical_accuracy: 0.6715
Epoch 5/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8155 - sparse_categorical_accuracy: 0.6279 - val_loss: 0.6843 - val_sparse_categorical_accuracy: 0.6715
Epoch 6/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8071 - sparse_categorical_accu

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁
precision,▁
recall,▁
seed,▁
test_accuracy,▁
train_accuracy,▁▅▆▇▆▆▆▅▇▅▆█▅█▆▆▆▆▇▆█▇▇▆▆▇▆▇▇▅▆██▇▇██▆▆█
train_loss,█▄▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▂▁▁▁▂▁▁▁▂▂▁
val_accuracy,▁▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇█▇██▇▇▇█
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99


In [4]:
def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [35]:
X_train = pd.read_pickle("../preprocess_clinical/X_train_c.pkl")
y_train = pd.read_pickle("../preprocess_clinical/y_train_c.pkl")

X_test = pd.read_pickle("../preprocess_clinical/X_test_c.pkl")
y_test = pd.read_pickle("../preprocess_clinical/y_test_c.pkl")

X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')

In [4]:
X_train.columns

Index(['NXVISUAL_-4.0', 'NXCONSCI_1.0', 'PTRACCAT_5', 'PTEDUCAT_13.0',
       'PTTLANG_2.0', 'NXPLANTA_2.0', 'PHC_EXF_PreciseFilter_1.0',
       'PHC_Visit_8', 'NXPLANTA_-4.0', 'PHC_MEM_PreciseFilter_1',
       ...
       'PTETHCAT_-4.0', 'PTEDUCAT_12.0', 'PHC_Visit_1', 'PTHAND_-4.0',
       'PTEDUCAT_-4.0', 'NXOTHER_-1.0', 'PTHOME_6.0', 'PTNOTRT_-4.0',
       'PTRACCAT_6', 'PHC_Age_Cognition'],
      dtype='object', length=149)

In [10]:
acc = []
f1 = []
precision = []
recall = []
seeds = random.sample(range(1, 200), 5)
for seed in seeds:
    reset_random_seeds(seed)
    model = Sequential()
    model.add(Dense(128, input_shape = (149,), activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    model.add(Dense(50, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Dense(3, activation = "softmax"))
    
    model.compile(Adam(learning_rate = 0.0001, decay=1e-6), "sparse_categorical_crossentropy", metrics = ["sparse_categorical_accuracy"])

    model.summary()
    history = model.fit(X_train, y_train,  epochs=100, validation_split=0.1, batch_size=32,verbose=1) 

    score = model.evaluate(X_test, y_test, verbose=0)
    print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
    acc.append(score[1])
    
    test_predictions = model.predict(X_test)
    test_label = to_categorical(y_test,3)

    true_label= np.argmax(test_label, axis =1)

    predicted_label= np.argmax(test_predictions, axis =1)
    
    cr = classification_report(true_label, predicted_label, output_dict=True)
    precision.append(cr["macro avg"]["precision"])
    recall.append(cr["macro avg"]["recall"])
    f1.append(cr["macro avg"]["f1-score"])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128)               19200     
                                                                 
 batch_normalization_6 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 batch_normalization_7 (Bat  (None, 64)                256       
 chNormalization)                                                
                                                                 
 dropout_7 (Dropout)         (None, 64)               

In [34]:
# reduce_lr
from tensorflow.keras.callbacks import ReduceLROnPlateau

wandb.init(project="Clinical", entity="yixuany")
# Configure wandb to log hyperparameters
config = wandb.config
config.epochs = 100
config.batch_size = 32
config.learning_rate = 0.001
config.decay = 1e-2
config.dropout = 0.2

# Set random seed
# seed = random.randint(1, 200)
seed = 177
wandb.log({"seed": seed})
reset_random_seeds(seed)

# Define the model
model = Sequential()
model.add(Dense(128, input_shape=(149,), activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(config.dropout))
model.add(Dense(128, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(config.dropout))
model.add(Dense(50, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(config.dropout))
model.add(Dense(3, activation="softmax"))

# Compile the model
model.compile(Adam(learning_rate=config.learning_rate, decay=config.decay),
              "sparse_categorical_crossentropy",
              metrics=["sparse_categorical_accuracy"])

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.6,
    patience=8,
    min_lr= 1e-10,
    verbose=1
)


# Custom callback to log metrics to wandb
class CustomWandbCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        wandb.log({
            "epoch": epoch,
            "train_loss": logs["loss"],
            "train_accuracy": logs["sparse_categorical_accuracy"],
            "val_loss": logs["val_loss"],
            "val_accuracy": logs["val_sparse_categorical_accuracy"]
        })

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=config.epochs,
    validation_split=0.1,
    batch_size=config.batch_size,
    verbose=1,
    callbacks=[CustomWandbCallback(), reduce_lr]
)

# Evaluate on test data
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# Make predictions and calculate metrics
test_predictions = model.predict(X_test)
test_label = to_categorical(y_test, 3)
true_label = np.argmax(test_label, axis=1)
predicted_label = np.argmax(test_predictions, axis=1)

# Calculate precision, recall, and F1-score
cr = classification_report(true_label, predicted_label, output_dict=True)

# Log final metrics to wandb
wandb.log({
    "test_accuracy": score[1],
    "precision": cr["macro avg"]["precision"],
    "recall": cr["macro avg"]["recall"],
    "f1_score": cr["macro avg"]["f1-score"]
})

wandb.finish()

Epoch 1/100
39/39 [==============================] - 1s 5ms/step - loss: 1.0009 - sparse_categorical_accuracy: 0.5743 - val_loss: 0.9981 - val_sparse_categorical_accuracy: 0.6715 - lr: 0.0010
Epoch 2/100
39/39 [==============================] - 0s 2ms/step - loss: 0.9135 - sparse_categorical_accuracy: 0.6198 - val_loss: 0.8210 - val_sparse_categorical_accuracy: 0.6715 - lr: 0.0010
Epoch 3/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8483 - sparse_categorical_accuracy: 0.6125 - val_loss: 0.7414 - val_sparse_categorical_accuracy: 0.6715 - lr: 0.0010
Epoch 4/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8160 - sparse_categorical_accuracy: 0.6247 - val_loss: 0.7144 - val_sparse_categorical_accuracy: 0.6715 - lr: 0.0010
Epoch 5/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8261 - sparse_categorical_accuracy: 0.6125 - val_loss: 0.7210 - val_sparse_categorical_accuracy: 0.6715 - lr: 0.0010
Epoch 6/100
39/39 [=====================

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁
precision,▁
recall,▁
seed,▁
test_accuracy,▁
train_accuracy,▁▃▃▄▃▃▄▄▅▅▅▆▇▆▆▆▆▆▆▆▇▇▇▆▇▇▇▇▇▇▇▇▇█▇█▇▇▇█
train_loss,█▅▄▄▄▄▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▂▁▂▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▃▆▆▄▃▆▆▆▁▆▆▆▆▆▇▇▆█▆▇▆▆▆▇█▇▇▇██▇▇▆██
val_loss,█▄▃▃▃▃▃▃▂▃▂▂▂▃▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
